In [11]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark import SQLContext


# !показать, что будет, если не сделать set+memory!
conf = SparkConf().setAppName("best_one").setMaster("spark://spark-master:7077").set("spark.executor.memory", "512m")
sc = SparkContext(conf=conf)


# spark = SparkSession.\
#         builder.\
#         appName("best_one").\
#         master("spark://spark-master:7077").\
#         config("spark.executor.memory", "512m").\
#         getOrCreate()



In [10]:
#sc.stop()

In [ ]:
#File("/data/shakespeare")

In [12]:
import re
from functools import reduce
from operator import add
from collections import Counter

# что внутри reduceByKey
from collections import OrderedDict
def reduce_by_key(ls):
    d = OrderedDict()
    for key, sublist in ls:
        d.setdefault(key, []).extend(sublist)
    return list(d.items())


def agg_list(lst):
    from collections import defaultdict
    agg = defaultdict(lambda : 0)
    for k, v in lst:
        agg[k] += v
    return list(agg.items())

In [13]:
# Inverted Index
sc.wholeTextFiles("data_shakespeare")\
.map(lambda x: (x[0].split("/")[-1], re.split('\W+', x[1])))\
.flatMap(lambda x: [(w.lower(),(x[0], 1)) for w in x[1]])\
.map(lambda x: (x[0], [x[1]]))\
.reduceByKey(lambda x,y: x + y)\
.groupByKey()\
.map(lambda x : (x[0], agg_list(list(x[1])[0])))\
.sortByKey()\
.take(10)


[('',
  [('asyoulikeit', 2),
   ('midsummersnightsdream', 2),
   ('muchadoaboutnothing', 2),
   ('comedyoferrors', 2),
   ('merrywivesofwindsor', 2),
   ('twelfthnight', 2),
   ('tamingoftheshrew', 2),
   ('loveslabourslost', 2)]),
 ('a',
  [('twelfthnight', 416),
   ('tamingoftheshrew', 445),
   ('loveslabourslost', 507),
   ('asyoulikeit', 461),
   ('midsummersnightsdream', 281),
   ('muchadoaboutnothing', 492),
   ('comedyoferrors', 254),
   ('merrywivesofwindsor', 494)]),
 ('abandon',
  [('twelfthnight', 1), ('tamingoftheshrew', 1), ('asyoulikeit', 4)]),
 ('abate',
  [('midsummersnightsdream', 1),
   ('tamingoftheshrew', 1),
   ('loveslabourslost', 1)]),
 ('abatement', [('twelfthnight', 1)]),
 ('abbess', [('comedyoferrors', 8)]),
 ('abbey', [('comedyoferrors', 9)]),
 ('abbominable', [('loveslabourslost', 1)]),
 ('abbreviated', [('loveslabourslost', 1)]),
 ('abed', [('asyoulikeit', 1), ('twelfthnight', 1)])]

In [14]:
# Inverted Index
sc.wholeTextFiles("data_shakespeare")\
.map(lambda x: (x[0].split("/")[-1], re.split('\W+', x[1])))\
.flatMap(lambda x: [(w.lower(),(x[0], 1)) for w in x[1] if len(w.lower()) >= 1])\
.map(lambda x: (x[0], [x[1]]))\
.reduceByKey(lambda x,y: x + y)\
.groupByKey()\
.map(lambda x : (x[0], agg_list(list(x[1])[0])))\
.sortByKey()\
.take(10)

# можно сделать элемент 2 строкой?

[('a',
  [('asyoulikeit', 461),
   ('midsummersnightsdream', 281),
   ('muchadoaboutnothing', 492),
   ('comedyoferrors', 254),
   ('merrywivesofwindsor', 494),
   ('twelfthnight', 416),
   ('tamingoftheshrew', 445),
   ('loveslabourslost', 507)]),
 ('abandon',
  [('asyoulikeit', 4), ('twelfthnight', 1), ('tamingoftheshrew', 1)]),
 ('abate',
  [('tamingoftheshrew', 1),
   ('loveslabourslost', 1),
   ('midsummersnightsdream', 1)]),
 ('abatement', [('twelfthnight', 1)]),
 ('abbess', [('comedyoferrors', 8)]),
 ('abbey', [('comedyoferrors', 9)]),
 ('abbominable', [('loveslabourslost', 1)]),
 ('abbreviated', [('loveslabourslost', 1)]),
 ('abed', [('twelfthnight', 1), ('asyoulikeit', 1)]),
 ('abetting', [('comedyoferrors', 1)])]

In [15]:
todf = sc.wholeTextFiles("data_shakespeare")\
.map(lambda x: (x[0].split("/")[-1], re.split('\W+', x[1])))\
.flatMap(lambda x: [(w.lower(),(x[0], 1)) for w in x[1] if len(w.lower()) >= 1])\
.map(lambda x: (x[0], [x[1]]))\
.reduceByKey(lambda x,y: x + y)\
.groupByKey()\
.map(lambda x : (x[0], agg_list(list(x[1])[0])))\
.sortByKey()

In [167]:
#sc.stop()

In [16]:
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

In [18]:
df = sqlContext.createDataFrame(todf).toDF("word", "locations_counts")

In [24]:
df.show(5, False)

+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|word     |locations_counts                                                                                                                                                                                        |
+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|a        |[[asyoulikeit, 461], [midsummersnightsdream, 281], [muchadoaboutnothing, 492], [comedyoferrors, 254], [merrywivesofwindsor, 494], [twelfthnight, 416], [tamingoftheshrew, 445], [loveslabourslost, 507]]|
|abandon  |[[asyoulikeit, 4], [twelfthnight, 1], [tamingoftheshrew, 1]]                                                                             

In [25]:
# explode
from pyspark.sql.functions import explode, col

df.select("word", explode(col("locations_counts")).alias("locations_counts")).show(5, False)

+----+----------------------------+
|word|locations_counts            |
+----+----------------------------+
|a   |[asyoulikeit, 461]          |
|a   |[midsummersnightsdream, 281]|
|a   |[muchadoaboutnothing, 492]  |
|a   |[comedyoferrors, 254]       |
|a   |[merrywivesofwindsor, 494]  |
+----+----------------------------+
only showing top 5 rows



In [26]:
df.printSchema()

root
 |-- word: string (nullable = true)
 |-- locations_counts: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _1: string (nullable = true)
 |    |    |-- _2: long (nullable = true)



In [ ]:
# Задание 1
# сделайте из этой таблцы: таблицу - "word", "total_count", "locations", "counts"
# "total_count" - это общее число элемента в документе

In [27]:
# Задание 2
# повторите скрипт, используя MAGIC_COMMANG / методы DataFrame:
# SELECT word, total_count, locations[0] AS top_location, counts[0] AS top_count 
# FROM inverted_index 

In [ ]:
# Задание 3
# Найдите слова `glove`, `gloves`, `whate` и `whatever` и найдите документ, где они чаще всего встречаются и кол-во упоминаний
# Ваш запрос должен возвращать колонки word, total_count, top_location,	top_count

In [ ]:
# Задание 4
# 1. Используйте список стоп слов и создайте новый DataFrame, без стоп слов
# 2. Выберите Топ-5 слов 
# 3. Сделайте новый DataFrame, имена колонок - слова из Топ-5 и total_count, а значением будет количество их встречаемости

# Финальный DF
# total_count | a | b
# ------------|---|--
#    1000     | 50| 50

In [28]:
# список стоп слов
stp = """
a
about
above
after
again
against
all
am
an
and
any
are
aren't
as
at
be
because
been
before
being
below
between
both
but
by
can't
cannot
could
couldn't
did
didn't
do
does
doesn't
doing
don't
down
during
each
few
for
from
further
had
hadn't
has
hasn't
have
haven't
having
he
he'd
he'll
he's
her
here
here's
hers
herself
him
himself
his
how
how's
i
i'd
i'll
i'm
i've
if
in
into
is
isn't
it
it's
its
itself
let's
me
more
most
mustn't
my
myself
no
nor
not
of
off
on
once
only
or
other
ought
our
ours	ourselves
out
over
own
same
shan't
she
she'd
she'll
she's
should
shouldn't
so
some
such
than
that
that's
the
their
theirs
them
themselves
then
there
there's
these
they
they'd
they'll
they're
they've
this
those
through
to
too
under
until
up
very
was
wasn't
we
we'd
we'll
we're
we've
were
weren't
what
what's
when
when's
where
where's
which
while
who
who's
whom
why
why's
with
won't
would
wouldn't
you
you'd
you'll
you're
you've
your
yours
yourself
yourselves"""

In [ ]:
df.groupby('') \
    .pivot('') \
    .max('') \
    .fillna(0)